In [ ]:
import sys
# Assuming we are in the notebook directory add this so that we can import the library
sys.path.append('..')

import numpy as np
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')
%matplotlib inline

# Inference with ELFI: the MA(2) model

The 2nd order moving average model, MA(2), is a common model used in univariate time analysis. Assuming zero mean it can be written as

$$
y_t = w_t + \theta_1 w_{t-1} + \theta_2 w_{t-2},
$$

where $\theta_1, \theta_2 \in \mathbb{R}$ and $(w_k)_{k\in \mathbb{Z}} \sim N(0,1)$ represents an independent and identically distributed sequence of white noise.

### The observed data and the inference problem

In this notebook, our task is to infer the parameters $\theta_1, \theta_2$ given a sequence of 100 observations $y$ that originate from an MA(2) process. Let's define this MA(2) simulator as a Python function:

In [ ]:
def MA2(t1, t2, batch_size=1, random_state=None):
    n_obs = 100
    if random_state is None:
        random_state = np.random.RandomState()
    w = random_state.randn(batch_size, n_obs+2) # i.i.d. sequence ~ N(0,1)
    y = w[:,2:] + t1 * w[:,1:-1] + t2 * w[:,:-2]
    return y

In ELFI, all non-constant data is in at least 2-dimensional NumPy arrays with different observations on the 0-axis. **Important**: in order to guarantee a consistent state of pseudo-random number generation, the simulator must have `random_state` as a keyword argument for reading in a `numpy.RandomState` object. Additionally, a simulator that supports vectorized operations should accept the keyword argument `batch_size` that defines how many sequences should be returned.

Let's now use this simulator to create the observations with true parameter values $\theta_1=0.6, \theta_2=0.2$ (from now on these are considered unknown):

In [ ]:
# true parameters
t1_0 = 0.6
t2_0 = 0.2

# Set up observed data y with some random seed
random_state = np.random.RandomState(20161130)
y = MA2(t1_0, t2_0, random_state=random_state)

# Plot the observed sequence
plt.figure(figsize=(11, 6));
plt.plot(y.flatten());

# To illustrate the stochasticity, let's plot a couple of more observations with the same true parameters:
plt.plot(MA2(t1_0, t2_0).flatten());
plt.plot(MA2(t1_0, t2_0).flatten());

### Approximate Bayesian Computation

The figure above illustrates the difficulty of inferring $\theta_1, \theta_2$. One way to approach this kind of problems is Approximate Bayesian Computation (ABC), which is based on the intuition that similar data is likely to have been produced by similar parameters. Although the idea may appear inapplicable for the task at hand, it works when a large number of samples can be used. For more information about ABC, please see e.g. 

* [Marin, J.-M., Pudlo, P., Robert, C. P., and Ryder, R. J. (2012). Approximate Bayesian computational
methods. *Statistics and Computing*, 22(6):1167–1180.](http://link.springer.com/article/10.1007/s11222-011-9288-2)
* [Lintusaari, J., Gutmann, M. U., Dutta, R., Kaski, S., and Corander, J. (2016). Fundamentals and recent
developments in approximate Bayesian computation. *Systematic Biology*, doi: 10.1093/sysbio/syw077.](http://sysbio.oxfordjournals.org/content/early/2016/09/07/sysbio.syw077.full.pdf)
* https://en.wikipedia.org/wiki/Approximate_Bayesian_computation

### Defining the inference problem in ELFI

In ELFI, the inference problem is described in the form of a directed acyclic graph ([DAG](https://en.wikipedia.org/wiki/Directed_acyclic_graph)), which associates each node with its parent nodes. This provides an intuitive means to describe complex dependencies that are automatically fulfilled by the inference engine. So let's build such a model.

As is usual in Bayesian statistical inference, we need to define *prior* distributions for the unknown parameters $\theta_1, \theta_2$. In ELFI the priors can be any of the continuous and discrete distributions available in `scipy.stats` (for custom priors, see [below](#custom_prior)). For simplicity, let's start by assuming that both parameters follow `Uniform(0, 2)`.

In [ ]:
import elfi

# a node is defined by giving it a name, a distribution from scipy.stats and its parents (here constants 0 and 2)
t1 = elfi.Prior('t1', scipy.stats.uniform, 0, 2)

# ELFI also supports giving the scipy.stats distributions as strings
t2 = elfi.Prior('t2', 'uniform', 0, 2)

Next, we define the *Simulator* node by giving it the `MA2` function, and the priors as its parents. As this node can be compared with observations, we give them as well.

In [ ]:
Y = elfi.Simulator('MA2', MA2, t1, t2, observed=y)

But how does one compare the simulations with the observed sequence? As was evident from the plot of just a few observed sequences, a direct pointwise comparison is unproductive. Indeed, the comparison of simulated sequences is often the most difficult (and arbitrary) part of ABC. Typically one chooses one or more *ad hoc* summary statistics and then calculates the discrepancy between those.

Here, we will apply the intuition arising from the definition of the MA(2) process, and use the autocovariances with lags 1 and 2 as the summary statistics, and evaluate the discrepancy with the common Euclidean L2-distance.

In [ ]:
def autocov(x, lag=1):
    mu = np.mean(x, axis=1, keepdims=True)
    C = np.mean(x[:,lag:] * x[:,:-lag], axis=1, keepdims=True) - mu**2.
    return C

def distance(x, y):
    d = np.linalg.norm( np.array(x) - np.array(y), ord=2, axis=0)
    return d

As is familiar by now, a `Summary` node is defined by giving the autocovariance function and the simulated data (which includes the observed as well):

In [ ]:
S1 = elfi.Summary('S1', autocov, Y)

The node syntax requires a function, so defining the second summary statistic as autocovariance with lag 2 is a bit more involved. Instead a writing a complete function from scratch, we can create a *closure* that fixes the keyword argument `lag`:

In [ ]:
from functools import partial

# this is a more efficient way to say `lambda x: autocov(x, lag=2)`
autocov2 = partial(autocov, lag=2)

S2 = elfi.Summary('S2', autocov2, Y)

# Finish the model with the final node that calculates the squared distance (S1_sim-S1_obs)**2 + (S2_sim-S2_obs)**2
d = elfi.Discrepancy('d', distance, S1, S2)

Now that the inference model is defined, ELFI can visualize the DAG:

In [ ]:
elfi.draw_model(d)

<a id='custom_prior'></a>


### Defining custom priors

Although the above definition is perfectly valid, having both prior distributions as `Uniform(0,2)` is bad, as the case becomes unidentifiable (i.e. the solution is not unique). To avoid this, *Marin et al. (2012)* defined the priors such that $-2<\theta_1<2$ with $\theta_1+\theta_2>-1$ and $\theta_1-\theta_2<1$ i.e. the parameters are sampled from a triangle (see below).

In ELFI, custom distributions can be defined as **static** classes (i.e. they only have static methods) that behave in a way similar to distributions from scipy.stats. To be safe they can inherit `elfi.Distribution`. In this case we only need these for sampling, so implementing the static `rvs` method suffices. As was in the context of simulators, it is important to accept the keyword argument `random_state`, which is needed for ELFI's internal book-keeping of pseudo-random number generation.

In [ ]:
# define prior for t1 as in Marin et al., 2012 with t1 in range [-b, b]
class MarinPrior_t1(elfi.Distribution):
    def rvs(b, size=1, random_state=None):
        u = scipy.stats.uniform.rvs(loc=0, scale=1, size=size, random_state=random_state)
        t1 = np.where(u<0.5, np.sqrt(2.*u)*b-b, -np.sqrt(2.*(1.-u))*b+b)
        return t1

# define prior for t2 conditionally on t1 as in Marin et al., 2012, in range [-a, a]
class MarinPrior_t2(elfi.Distribution):
    def rvs(t1, a, size=1, random_state=None):
        locs = np.maximum(-a-t1, t1-a)
        scales = a - locs
        t2 = scipy.stats.uniform.rvs(loc=locs, scale=scales, size=size, random_state=random_state)
        return t2

These indeed sample from a triangle:

In [ ]:
t1_1000 = MarinPrior_t1.rvs(2, 1000)
t2_1000 = MarinPrior_t2.rvs(t1_1000, 1, 1000)
plt.scatter(t1_1000, t2_1000, s=4, edgecolor='none');

Let's change the earlier priors to the new ones in the inference model:

In [ ]:
# Redefine the priors, since t1 and t2 already exist
t1.redefine(MarinPrior_t1, 2)
t2.redefine(MarinPrior_t2, t1, 1)

elfi.draw_model(d)

### Basic inference with rejection sampling

The simplest ABC algorithm samples parameters from their prior distributions, runs the simulator with these and compares to the observations. The samples are either accepted or rejected depending on how large the discrepancy is. The accepted samples represent samples from the posterior distribution.

In ELFI, ABC methods are initialized with the discrepancy node and a list of inferred parameters (in Python lists are defined with square brackets). The optional keyword argument `batch_size` is related to parallelization (see the chapter on [parallelization](#parallelization) below).

In [ ]:
rej = elfi.Rejection(d, [t1, t2], batch_size=10000)

After the ABC method has been initialized, samples can be drawn from it. By default, rejection sampling in ELFI works in `quantile` mode i.e. a certain quantile of the samples with smallest discrepancies is accepted. The `sample` method requires the number of output samples as a parameter. Note that the simulator is then run `(N/quantile)` times.

The `sample` method returns a `Result` object, which contains several attributes and methods. For example the attribute 'samples' contains an OrderedDict of the posterior numpy arrays. For rejection sampling, other attributes include e.g. the 'threshold', which is the threshold value resulting in the requested quantile.

In [ ]:
# Time and run the simulator
N = 10000
%time result = rej.sample(N, quantile=0.01)

print(result)

Rejection sampling can also be performed `threshold` based i.e. accepting all samples that result in a discrepancy below certain threshold. Note that since we require a fixed number of samples, there is no guarantee how many times the simulator will be run.

In [ ]:
%time result2 = rej.sample(N, threshold=0.2)

print(result2)

The rejection sampler also has a method for reconsidering existing simulations with a potentially new threshold. By default it uses all existing simulations, but one may also choose to use more or less. With the `reject` method the threshold is fixed as is the number of simulations.

In [ ]:
%time result3 = rej.reject(threshold=0.3)

print(result3)

### Visualizing inference

Instances of `Result` contain methods for some basic plotting (these are convenience methods to plotting functions defined in `elfi.visualization`).

For example one can plot the marginal distributions:

In [ ]:
result.plot_marginals();

Often "pairwise relationships" are more informative:

In [ ]:
result.plot_pairs();

### Automatic parallelization

Behind the scenes, ELFI uses the [Dask](http://dask.pydata.org/) library to automatically parallelize (most of) the computational inference. The parallelization can be tuned with the keyword argument `batch_size` for the Rejection class, which tells how many "runs"$^1$ should be sent to each available computational unit at a time. There is some overhead involved in the parallelization, so batches should be large, but not too large to eat all your memory.

$^1$To be more accurate, the MA2 model as well as the other functions in this example are vectorized, and the simulator is actually called just a few times with the keyword argument `n_sim` set to `batch_size`. This results in more efficient usage of numpy operations. Whenever possible, functions should be written in a form that allows vectorization.

The underlying Dask graph for our inference can be visualized (partly):

In [ ]:
from dask.dot import dot_graph
dot_graph(d[0:40000].dask)

### Sequential Monte Carlo ABC

Rejection sampling is quite inefficient, as it does not learn from its history. The sequential Monte Carlo (SMC) ABC algorithm does just that by applying importance sampling: samples are *weighed* according to the resulting discrepancies and the next *population* of samples is drawn near to the previous using the weights as probabilities. 

For evaluating the weights, SMC ABC needs to have probability density functions for the priors. In our MA2 example the second prior is conditional on the first, which complicates matters a bit. Let's modify the prior distribution classes:

In [ ]:
# define prior for t1 as in Marin et al., 2012 with t1 in range [-b, b]
class MarinPrior_t1(elfi.Distribution):
    def rvs(b, size=1, random_state=None):
        u = scipy.stats.uniform.rvs(loc=0, scale=1, size=size, random_state=random_state)
        t1 = np.where(u<0.5, np.sqrt(2.*u)*b-b, -np.sqrt(2.*(1.-u))*b+b)
        return t1
    
    def pdf(x, b):
        p = 1./b - np.abs(x) / (b*b)
        p = np.where(p < 0., 0., p)  # disallow values outside of [-b, b] (affects weights only)
        return p

# define prior for t2 conditionally on t1 as in Marin et al., 2012, in range [-a, a]
class MarinPrior_t2(elfi.Distribution):
    def rvs(t1, a, size=1, random_state=None):
        locs = np.maximum(-a-t1, t1-a)
        scales = a - locs
        t2 = scipy.stats.uniform.rvs(loc=locs, scale=scales, size=size, random_state=random_state)
        return t2
    
    def pdf(x, t1, a):
        locs = np.maximum(-a-t1, t1-a)
        scales = a - locs
        p = scipy.stats.uniform.pdf(x, loc=locs, scale=scales)
        p = np.where(scales>0., p, 0.)  # disallow values outside of [-a, a] (affects weights only)
        return p
    
# Redefine the priors
t1.redefine(MarinPrior_t1, 2)
t2.redefine(MarinPrior_t2, t1, 1)

In ELFI, one can setup a SMC ABC sampler just like the Rejection sampler:

In [ ]:
smc = elfi.SMC(d, [t1, t2], batch_size=10000)

For sampling, one has to define the number of output samples, the number of populations and a *schedule* i.e. a list of quantiles to use for each population. In essence, a population is just refined rejection sampling.

In [ ]:
N = 1000
n_populations = 3
schedule = [0.7, 0.2, 0.05]
%time result_dict = smc.sample(N, n_populations, schedule)

Let's plot the results:

In [ ]:
fig, ax = plt.subplots(ncols=n_populations, sharex=True, sharey=True, figsize=(16,6))

samples = result_dict['samples_history'] + [result_dict['samples']]
weights = result_dict['weights_history'] + [result_dict['weights']]

for ii in range(n_populations):
    s = samples[ii]
    w = weights[ii]
    print("Posterior means for population {}: t1: {:.2f} t2: {:.2f}".format(ii+1,\
                                                                            np.average(s[0], weights=w, axis=0)[0],\
                                                                            np.average(s[1], weights=w, axis=0)[0]))
    ax[ii].scatter(s[0], s[1], s=1, edgecolor='none');
    ax[ii].set_title("Population {}".format(ii+1));

It can be seen that the populations iteratively concentrate more and more around the true parameter values.

Note that for the later populations some of the samples lie outside allowed region. This is due to the SMC algorithm sampling near previous samples, with *near* meaning a Gaussian distribution centered around previous samples with variance as twice the weighted empirical variance. However, the outliers carry zero weight, as was defined in the prior pdfs.

### Bayesian Optimization for Likelihood-Free Inference (BOLFI)

In practice inference problems often have a more complicated and computationally heavy simulator than the function `MA2` here, and one simply cannot run it for millions of times. The [BOLFI](https://arxiv.org/abs/1501.03291) framework is likely to prove useful in such situation: a statistical model (e.g. [Gaussian process](https://en.wikipedia.org/wiki/Gaussian_process)) is created for the discrepancy, and its minimum is inferred with [Bayesian optimization](https://en.wikipedia.org/wiki/Bayesian_optimization). This approach typically reduces the number of required simulator calls by several orders of magnitude.

As BOLFI is more advanced inference method, its interface is also a bit more involved. But not much: Using the same graphical model as earlier, the inference begins by defining a Gaussian process model, for which we use the [GPy](https://sheffieldml.github.io/GPy/) library. We are inferring 2 parameters with the same bounds as earlier, and we have to give these conditions to the surrogate model i.e. the Gaussian process.

In [ ]:
gp_model = elfi.GPyModel(input_dim=2, bounds=((-3,3), (-3,3)))

In [ ]:
# redefine the model
elfi.env.new_inference_task()
t1 = elfi.Prior('t1', scipy.stats.uniform, 0, 2)
t2 = elfi.Prior('t2', 'uniform', 0, 2)
Y = elfi.Simulator('MA2', MA2, t1, t2, observed=y)
S1 = elfi.Summary('S1', autocov, Y)
S2 = elfi.Summary('S2', autocov2, Y)
d = elfi.Discrepancy('d', distance, S1, S2)

Taking a logarithm of the discrepancies reduces the effect that high discrepancies have on the GP.

In [ ]:
log_distance = lambda s1,s2: np.log(distance(s1, s2))
d = d.change_to(elfi.Discrepancy('log_d', log_distance, S1, S2), transfer_parents=True)

Having defined the surrogate model, we can instantiate the BOLFI framework object in a somewhat similar way as earlier, except that we now additionally define the surrogate model and the number of samples to take from it (this is the number of simulator calls): 

In [ ]:
bolfi = elfi.BOLFI(d, [t1, t2], batch_size=5, n_surrogate_samples=150, model=gp_model)

The BOLFI class can now try to `infer` the posterior distribution of the parameters:

In [ ]:
post = bolfi.infer()

In [ ]:
gp_model.gp

We can get estimates for *maximum a posteriori* and *maximum likelihood* easily:

In [ ]:
post.MAP, post.ML

Discrepancies for samples:

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(16,6), sharey=True)
ax[0].scatter(bolfi.model.gp.X[:,0], bolfi.model.gp.Y[:,0]);
ax[1].scatter(bolfi.model.gp.X[:,1], bolfi.model.gp.Y[:,0]);
ax[0].set_ylabel('Discrepancy');
ax[0].set_xlabel('t1');
ax[1].set_xlabel('t2');

We can also plot the Gaussian process using GPy:

In [ ]:
bolfi.model.gp.plot();

... or we can visualize the posterior directly:

In [ ]:
x, y = np.meshgrid(np.linspace(*bolfi.model.bounds[0]), np.linspace(*bolfi.model.bounds[1]))
z = np.empty_like(x)
for ii in range(len(x)):
    for jj in range(len(x)):
        z[ii, jj] = post.pdf(np.array([x[ii, jj], y[ii, jj]]))
plt.contour(x, y, z);

### Saving samples for later use

#### This section is still under construction!

Saving performed computations is often a good idea, especially if the simulator is expensive to call. ELFI supports persistence of results in multiple formats. The user simply has to define this with the `store` keyword argument, which can be given to all nodes in the graphical model upon instantiation.

To cache the prior `t1` in memory (affects only `t1`):

In [ ]:
# t1 already exists
elfi.env.new_inference_task()
t1 = elfi.Prior('t1', 'uniform', 0, 1, store="cache")

To store directly in a Numpy array:

In [ ]:
max_samples = 10000000
t1_array = np.empty((max_samples, 1))  # make sure you allocate enough memory
elfi.env.new_inference_task()
t1 = elfi.Prior('t1', 'uniform', 0, 1, store=t1_array)

A NoSQL database is also supported, in which case the results are saved to collection matching node name:

In [ ]:
nosql_store = elfi.UnQLiteStore()  # accepts filename to save to disk
elfi.env.new_inference_task()
Y = elfi.Simulator('MA2', MA2, t1, t2, observed=y, store=nosql_store)

# afterwards get data: